In [1]:
import time
from typing import Dict, List
import pandas as pd
import numpy as np
import datetime
import glob

STOREPath =  "../data/factorData/"
OHLCPath = "../data/lakeAPIData/OHLC/"

# Use a custom list
# TARGET = ['BNB-USDT', 'BTC-USDT', 'ETH-USDT', 'XRP-USDT']

# Dynamically create the list from file names
fileLists = glob.glob(OHLCPath + "*")
TARGET = [file.split("/")[-1].split(".")[0] for file in fileLists]

fileLists = glob.glob(OHLCPath + "*")


def extractDateFromTime(timestamp:pd.Timestamp)->datetime.date:
    year = timestamp.year
    month = timestamp.month
    day = timestamp.day
    date = datetime.datetime(year=year, month=month, day=day)
    return date 

data = []

for file in fileLists:

    minuteLevelOHLC = pd.read_csv(file, compression="gzip")
    minuteLevelOHLC = minuteLevelOHLC.convert_dtypes()
    minuteLevelOHLC['origin_time'] = pd.to_datetime(minuteLevelOHLC['origin_time'])
    minuteLevelOHLC['date'] = minuteLevelOHLC['origin_time'].apply(lambda x : extractDateFromTime(x))
    data += [minuteLevelOHLC]

data = pd.concat(data)
data['return'] = (data['close'] - data['open'])/data['open']
#Market return is defined as the average return of all the cryptocurrencies return
mktReturn = data.groupby("origin_time").agg({'return':np.mean})
mktReturn.rename(columns={'return':'mktReturn'}, inplace=True)
data = pd.merge(data, mktReturn, on='origin_time', how='left')
data = data.dropna()


/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/3164020738.py:41: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  mktReturn = data.groupby("origin_time").agg({'return':np.mean})


In [2]:
TOTALNUMOFCOINS = len(set(data['symbol'].values))
LOWLEVEL = int(TOTALNUMOFCOINS/3)
HIGHLEVEL= int(TOTALNUMOFCOINS - TOTALNUMOFCOINS/3)

In [3]:
def generateLiqBucketSymbols(data:pd.DataFrame)->Dict:
    """"""
    Liq = data.groupby("symbol").agg({'trades':np.mean}).sort_values(by='trades')
    Liq.rename(columns={'trades':'Liq'}, inplace=True)
    LOWLiq = Liq['Liq'].values[0:LOWLEVEL].max()
    HIGHLiq = Liq['Liq'].values[0:HIGHLEVEL].max()
    lowLiqSymbols = Liq[Liq['Liq']<=LOWLiq].reset_index()['symbol'].values
    mediumLiqSymbols = Liq[(Liq['Liq']>LOWLiq)&(Liq['Liq']<HIGHLiq)].reset_index()['symbol'].values
    highLiqSymbols = Liq[Liq['Liq']>=HIGHLiq].reset_index()['symbol'].values
    return {'low': lowLiqSymbols, 'medium':mediumLiqSymbols, 'high':highLiqSymbols}

LiqBucketSyms = generateLiqBucketSymbols(data)

/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/3442124573.py:3: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  Liq = data.groupby("symbol").agg({'trades':np.mean}).sort_values(by='trades')


In [4]:
def getBeta(xVar:str, yVar:str, df:pd.DataFrame) -> float:
    df = df.sort_values("received_time").set_index("received_time")
    x = df[xVar].values
    y = df[yVar].values
    beta = np.sum( (x - np.mean(x))*(y - np.mean(y)) ) / np.sum((x - np.mean(x))**2)
    return beta 


Beta = data.groupby("symbol").apply(lambda x : getBeta('return','mktReturn', x))
Beta = pd.DataFrame({'beta':Beta})

def generateVolBucketSymbols(Beta:pd.DataFrame) ->Dict:
    Beta = Beta.sort_values(by='beta')
    LOWBeta = Beta['beta'].values[0:LOWLEVEL].max()
    HIGHBeta = Beta['beta'].values[0:HIGHLEVEL].max()
    lowBetaSymbols = Beta[Beta['beta']<=LOWBeta].reset_index()['symbol'].values
    mediumBetaSymbols = Beta[(Beta['beta']>LOWBeta)&(Beta['beta']<HIGHBeta)].reset_index()['symbol'].values
    highBetaSymbols = Beta[Beta['beta']>=HIGHBeta].reset_index()['symbol'].values
    return {'low':lowBetaSymbols, 'medium':mediumBetaSymbols, 'high':highBetaSymbols}

VolBucketSyms = generateVolBucketSymbols(Beta)

In [5]:
import functools

def getMomentum(basicRet:str, df:pd.DataFrame, windows:List = [60,2*60, 3*60,6*60,12*60]) -> float:
    df = df.sort_values(by='received_time')
    for window in windows:
        df[basicRet+str(window)] = df.rolling(window)['return'].sum()
    df['momentum'] = functools.reduce(lambda a, b: a+b, [df[basicRet+str(window)] for window in windows])

    return df 

momentum = data.groupby("symbol").apply(lambda x: getMomentum('return',x)).reset_index(drop=True)
momentum = momentum.groupby(["symbol","date"]).agg({'momentum':np.mean}).reset_index()
#last day momentum
momentum = momentum[momentum['date'] == '2024-01-30']

def generateMomBucketSymbols(Mom:pd.DataFrame, date:str)->Dict:
    Mom = Mom[Mom['date']==date].sort_values(by='momentum')
    LOWMom = Mom['momentum'].values[0:LOWLEVEL].max()
    HIGHMom = Mom['momentum'].values[0:HIGHLEVEL].max()
    lowMomSymbols = Mom[Mom['momentum']<=LOWMom].reset_index()['symbol'].values
    mediumMomSymbols = Mom[(Mom['momentum']>LOWMom)&(Mom['momentum']<HIGHMom)].reset_index()['symbol'].values
    highMomSymbols = Mom[Mom['momentum']>=HIGHMom].reset_index()['symbol'].values
    return {'low':lowMomSymbols, 'medium':mediumMomSymbols, 'high':highMomSymbols}

MomBucketSyms = generateMomBucketSymbols(momentum,'2024-01-30')

/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/4035521881.py:12: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  momentum = momentum.groupby(["symbol","date"]).agg({'momentum':np.mean}).reset_index()


In [6]:
class riskFactor(object):
    
    def __init__(self, name, componentSymbols, freq = '1m'):
        self._factorName = name
        self._symbols = componentSymbols
        self._freq = '1m'
        return 
    
    @property
    def name(self):
        return self._factorName
    
    @property
    def symbols(self):
        return self._symbols
        
    @property
    def freq(self):
        return self._freq
    
    @property
    def factorRetTS(self):
        return self._factorTS
    
    def getFactorReturnTS(self, data:pd.DataFrame)->pd.DataFrame:
        """
        Calculate risk factor returns given frequency and symbols
        Parameters
        ----------
        data: a panel dataframe with all symbols and their corresponding returns

        Returns: a timeSeries table of risk factor returns and time
        -------

        """
        factorRet = data[data['symbol'].isin(self._symbols)].groupby("origin_time").agg({'return':np.mean})
        factorRet.rename(columns={'return':self._factorName}, inplace=True)
        self._factorTS = factorRet
        return 
        
    
    def storeData(self):
        self._factorTS.to_csv(STOREPath+self._factorName+".csv", index = False)
        

class cryptoAsset(object):
    
    def __init__(self, symbol:str, sourceData:pd.DataFrame):
        self._symbol = symbol 
        self._sourceData = sourceData
        
    @property
    def symbol(self):
        return self._symbol
    
    @property
    def retTs(self):
        data = self._sourceData[self._sourceData['symbol'] == self._symbol][['date','origin_time','symbol','return']]
        return data
    
        
BNB  = cryptoAsset('BNB-USDT', data)
BTC  = cryptoAsset('BTC-USDT', data)
ETH  = cryptoAsset('ETH-USDT', data)
XRP  = cryptoAsset('XRP-USDT', data)


## Systematic Risk Factors

>* Liquidity: High,Medium, Low, liquidity is defined as the average trading volume
>* Momentum: High, Medium, Low, momentum is defined as the crypto's past 3 hour, 6 hours, 12 hours, 18 hours
>* Market: Average crypto coins return
>* Volatility: High, Medium, Low, defined as the beta coefficient to the market risk

In [7]:
marketRisk = riskFactor("market", list(set(data['symbol'].values)))
marketRisk.getFactorReturnTS(data)
marketRisk.storeData()
#Liquidity Risk Bucket
lowLiqRisk = riskFactor("Liq:low", LiqBucketSyms['low'])
lowLiqRisk.getFactorReturnTS(data)
lowLiqRisk.storeData()

medLiqRisk = riskFactor("Liq:med", LiqBucketSyms['medium'])
medLiqRisk.getFactorReturnTS(data)
medLiqRisk.storeData()

highLiqRisk = riskFactor("Liq:high", LiqBucketSyms['high'])
highLiqRisk.getFactorReturnTS(data)
highLiqRisk.storeData()
#Volatility Risk Bucket
lowVolRisk = riskFactor("Vol:low", VolBucketSyms['low'])
lowVolRisk.getFactorReturnTS(data)
lowVolRisk.storeData()

medVolRisk = riskFactor("Vol:med", VolBucketSyms['medium'])
medVolRisk.getFactorReturnTS(data)
medVolRisk.storeData()

highVolRisk = riskFactor("Vol:high", VolBucketSyms['high'])
highVolRisk.getFactorReturnTS(data)
highVolRisk.storeData()
#Momentum Risk Bucket
lowMomRisk = riskFactor("Mom:low", MomBucketSyms['low'])
lowMomRisk.getFactorReturnTS(data)
lowMomRisk.storeData()

medMomRisk = riskFactor("Mom:med", MomBucketSyms['medium'])
medMomRisk.getFactorReturnTS(data)
medMomRisk.storeData()

highMomRisk = riskFactor("Mom:high", MomBucketSyms['high'])
highMomRisk.getFactorReturnTS(data)
highMomRisk.storeData()

/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/1075514776.py:36: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  factorRet = data[data['symbol'].isin(self._symbols)].groupby("origin_time").agg({'return':np.mean})
/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/1075514776.py:36: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  factorRet = data[data['symbol'].isin(self._symbols)].groupby("origin_time").agg({'return':np.mean})
/var/folders/v7/9lvm55pn1fjf8b5b1vksfnwh0000gn/T/ipykernel_64432/1075514776.py:36: FutureWarning: The provided callable <function mean at 0x104bf9940> is currently using Se

In [8]:
from numpy.linalg import inv
import os

class factorRiskModel(object):
    
    def __init__(self, riskFactors:List[riskFactor], coin:cryptoAsset ):
        self._coin = coin
        self._riskFactors = riskFactors
        self._factorNames = None
    
    @property
    def symbol(self):
        return self._coin.symbol
    
    @property
    def riskFactors(self):
        factors = []
        for rf in self._riskFactors:
            factors += [rf.name]
        return factors
    
    def rollingReg(self, df:pd.DataFrame, xvar:List[str], yvar:str, window:int)->pd.DataFrame:
        from numpy.linalg import inv
        Y = df[yvar].rolling(window)
        X = df[xvar].rolling(window)
        
        i = 1 
        for x,y in zip(X,Y):
            
            x = x.values.astype('float').reshape(-1,len(xvar))
            ones = np.ones((x.shape[0],1))
            x = np.concatenate([ones, x], axis = 1).reshape(-1,len(xvar) + 1)
            y = y.values.astype('float').reshape(-1,1)

            coefs = np.dot(inv(np.dot(x.T,x)),np.dot(x.T,y))
            # reg_param = 1e-6
            # coefs = np.dot(inv(np.dot(x.T, x) + reg_param * np.eye(x.shape[1])), np.dot(x.T, y))
            Yhat = np.dot(x, coefs)
            yMean = np.mean(y)
            Rsqrd = 1 - np.sum((y-Yhat)**2)/np.sum((y - yMean)**2)
            RsqrdAdj = 1 - (1-Rsqrd)*(x.shape[0] - 1)/(x.shape[0] - 1 - len(xvar))
            df.loc[window + i, 'Rsqrd'] = Rsqrd
            df.loc[window + i, 'RsqrdAdj'] = RsqrdAdj
            for j in range(len(coefs)):
                if j == 0:
                   df.loc[window+i,'constant'] = coefs[j][0]
                else:
                    df.loc[window+i,xvar[j-1]+'_coef'] = coefs[j][0]
            
            i += 1
        return df 

    def calcRiskExposure(self, window=720)->pd.DataFrame:
        """
        calculate a rolling time series regression to get the dynamic risk exposure per coin to systematic risk factors
        Parameters
        ----------
        window

        Returns a time series of risk exposures
        -------

        """
        data = self._coin.retTs
        for rf in self._riskFactors:
            data = pd.merge(data, rf.factorRetTS, on = 'origin_time', how = 'left')
        
        #1440=60*24 hour rolling risk model
        rollFactorExposure = self.rollingReg(data.dropna(), self.riskFactors, 'return', 1440).dropna()

        return rollFactorExposure
    
    def saveRiskExposure(self, exposure_df: pd.DataFrame, file_path: str):
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            os.makedirs(directory)
        exposure_df.to_csv(file_path, index=False)

    
riskFactors = [marketRisk,lowLiqRisk,medLiqRisk,highLiqRisk,lowVolRisk,medVolRisk,highVolRisk,lowMomRisk,medMomRisk,highMomRisk]



In [9]:
#get risk factor exposure and R2 and adj R2 

# Calculate factor exposures for each asset
fmBTC = factorRiskModel(riskFactors, BTC)
BTCFactorExposure = fmBTC.calcRiskExposure()
fmBTC.saveRiskExposure(BTCFactorExposure, '../data/factorData/exposures/BTCFactorExposure.csv')

fmBNB = factorRiskModel(riskFactors, BNB)
BNBFactorExposure = fmBNB.calcRiskExposure()
fmBNB.saveRiskExposure(BNBFactorExposure, '../data/factorData/exposures/BNBFactorExposure.csv')

fmETH = factorRiskModel(riskFactors, ETH)
ETHFactorExposure = fmETH.calcRiskExposure()
fmETH.saveRiskExposure(ETHFactorExposure, '../data/factorData/exposures/ETHFactorExposure.csv')

fmXRP = factorRiskModel(riskFactors, XRP)
XRPFactorExposure = fmXRP.calcRiskExposure()
fmXRP.saveRiskExposure(XRPFactorExposure, '../data/factorData/exposures/XRPFactorExposure.csv')


LinAlgError: Singular matrix